## Модель Фицхью-Нагумо
![FNModel](modelFN.png)

![FTauShapes](f_tau_shapes.png)

In [1]:
# константы кусочно-линейной функции f(E)
C1 = 4
C2 = 1
C3 = 15

# константы кусочно-константной функции tau(E)
tau1 = 0.5
tau2 = 16.66
tau3 = 0.5

# границы кусочности функции f(E)
E1 = 0.018
E2 = ((C1 + C2) * E1 + C3) / (C3 + C2)

a = E1 * (C1 / C2)

# границы кусочности функции tau(E)
B1 = 0.01
B2 = 0.95

In [2]:
# Функция f(E)
def f(E):
    return -C1 * E if E < E1 else C2 * (E - a) if E1 <= E <= E2 else -C3 * (E - 1)

In [4]:
# Функция tau(E)
def tau(E):
    return tau1 if E < B1 else tau2 if B1 <= E <= B2 else tau3

In [5]:
import numpy as np
step = 1.2
x_max = 72
x = np.arange(0.0, x_max, step)
y_max = 48
y = np.arange(0.0, y_max, step)